## ADSA SGD Task 2: Digit Recogniser

In this task we will take images of numbers 1 to 9, and attempt to create an algorithm to classify each number. 

Helpful information can be found at the following link: 
https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6

The basic structure is as follows
1. Introduction

2. Data preparation
<br>2.1 Load data
<br>2.2 Check for null and missing values
<br>2.3 Normalization
<br>2.4 Reshape
<br>2.5 Label encoding
<br>2.6 Split training and valdiation set

3. CNN
<br>3.1 Define the model
<br>3.2 Set the optimizer and annealer
<br>3.3 Data augmentation

4. Evaluate the model
<br>4.1 Training and validation curves
<br>4.2 Confusion matrix

5. Prediction and submition
<br>5.1 Predict and Submit results

## Intro


We are going to use the Keras library, with 2 epochs. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')